# Fibonacci Runner

This notebook demonstrates how to create a grading workflow using PyBryt.

In [1]:
import pybryt

This demo has the following directory structure. This notebook, `index.ipynb`, runs PyBryt, `fibonacci.ipynb` is the assignment reference implementation, and `submissions` contains notebooks with student code in them.

In [2]:
%%bash
tree

.
├── fibonacci.ipynb
├── fibonacci_dyn.pkl
├── fibonacci_map.pkl
├── fibonacci_no_recurse.pkl
├── index.ipynb
└── submissions
    ├── subm01.ipynb
    ├── subm02.ipynb
    └── subm03.ipynb

1 directory, 8 files


## Reference Implementations

If you have marked up a reference implementation, like the one in [`fibonacci.ipynb`](fibonacci.ipynb), you can load this reference using `pybryt.ReferenceImplementation.compile`. Because references are relatively static and can take some time to execute, you can pickle the reference implementations to a file with `pybryt.ReferenceImplementation.dump`. Because this notebook has multiple references, each needs to be pickled separately.

In [9]:
refs = pybryt.ReferenceImplementation.compile("fibonacci.ipynb")
for ref in refs:
    ref.dump()

0 1 1 2 3 5 8 13 21 34 0 1 1 2 3 5 8 13 21 34 

## Assessing Submissions

To use PyBryt for grading multiple submissions, you can build a reproducible grading pipeline for an arbitrary number of submissions. To grab the submission notebook paths, the cell below uses `glob.glob`.

In [4]:
from glob import glob
subms = sorted(glob("submissions/*.ipynb"))
subms

['submissions/subm01.ipynb',
 'submissions/subm02.ipynb',
 'submissions/subm03.ipynb']

To use PyBryt to grade a student's submission, a `pybryt.StudentImplementation` must be created from that submission. The constructor takes the path to the notebook as its only positional argument. It is in this step that the student's code is executed, so this cell will need to be rerun whenever changes are made to the submission notebooks.

In [5]:
%%time
student_impls = []
for subm in subms:
    print(f"Executing {subm}")
    student_impls.append(pybryt.StudentImplementation(subm))

student_impls

Executing submissions/subm01.ipynb
Executing submissions/subm02.ipynb
Executing submissions/subm03.ipynb
CPU times: user 162 ms, sys: 89.9 ms, total: 252 ms
Wall time: 1min 30s


Once you have created the `pybryt.StudentImplementation` objects, use the `pybryt.StudentImplementation.check` method to run the check of a submission against a reference implementation. This method returns a single `pybryt.ReferenceResult` or a list of them, depending on the argument passed to `check`. In the cell below, the results are collected into a list.

In [10]:
results = []
for si in student_impls:
    results.append(si.check(refs))

results

[[ReferenceResult([
    AnnotationResult(satisfied=True, annotation=pybryt.Value),
    AnnotationResult(satisfied=True, annotation=pybryt.Value),
    AnnotationResult(satisfied=True, annotation=pybryt.Value),
    AnnotationResult(satisfied=True, annotation=pybryt.Value),
    AnnotationResult(satisfied=True, annotation=pybryt.Value),
    AnnotationResult(satisfied=True, annotation=pybryt.Value),
    AnnotationResult(satisfied=True, annotation=pybryt.Value),
    AnnotationResult(satisfied=True, annotation=pybryt.Value),
    AnnotationResult(satisfied=True, annotation=pybryt.TimeComplexity)
  ]),
  ReferenceResult([
    AnnotationResult(satisfied=False, annotation=pybryt.Value),
    AnnotationResult(satisfied=False, annotation=pybryt.Value),
    AnnotationResult(satisfied=False, annotation=pybryt.Value),
    AnnotationResult(satisfied=False, annotation=pybryt.Value),
    AnnotationResult(satisfied=False, annotation=pybryt.Value),
    AnnotationResult(satisfied=False, annotation=pybryt.Val

To view the results in a concise manner, the `pybryt.ReferenceResult` class has some helpful instance variables. You can also get information about the memory footprint, such as the number of steps, from the `pybryt.StudentImplementation` class.

In [11]:
from textwrap import indent
for sp, si, res in zip(subms, student_impls, results):
    print(f"SUBMISSION: {sp}")
    print(f"  EXECUTION STEPS: {si.steps}") # the number of steps in the execution
    
    # generate a summary report
    report = indent(pybryt.generate_report(res), "  ")
    print(report)

    print("\n")

SUBMISSION: submissions/subm01.ipynb
  EXECUTION STEPS: 801
  REFERENCE: fibonacci_map
  SATISFIED: True
  MESSAGES:
    - Found hash map implementation
    - Runs in linear time

  REFERENCE: fibonacci_dyn
  SATISFIED: False
  MESSAGES:
    - Runs in linear time

  REFERENCE: fibonacci_no_recurse
  SATISFIED: True
  MESSAGES:
    - Found hash map implementation


SUBMISSION: submissions/subm02.ipynb
  EXECUTION STEPS: 2233
  REFERENCE: fibonacci_map
  SATISFIED: False
  MESSAGES:
    - Runs in linear time

  REFERENCE: fibonacci_dyn
  SATISFIED: True
  MESSAGES:
    - Found dynamic programming implementation
    - Runs in linear time

  REFERENCE: fibonacci_no_recurse
  SATISFIED: True
  MESSAGES:
    - Found dynamic programming implementation


SUBMISSION: submissions/subm03.ipynb
  EXECUTION STEPS: 31844337
  REFERENCE: fibonacci_map
  SATISFIED: False
  MESSAGES:
    - ERROR: Does not run in linear time

  REFERENCE: fibonacci_dyn
  SATISFIED: False
  MESSAGES:
    - ERROR: Does no